In [0]:
#set up
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# from utils import policy_dict, policy_data_dict, find_non_zero_riskweight_rules, find_zero_riskweight_rules
from pandasql import sqldf
from tqdm import tqdm
import logging

# Standard data manipulation and visualization packages
import pandas as pd
import numpy as np



# Display settings
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show 100 rows max
pd.set_option('display.width', None)        # Auto-detect display width
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Format floats to 3 decimal places

# Plotting settings

plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 6]  # Set default figure size


from pysnowflake import Session
run_query = lambda query: sqldf(query, globals())

user_name = 'jobyg' #replace it with your ldap name



In [0]:
user_name = 'jobyg' #replace it with your ldap name
sess = Session(
   connection_override_args={
       'autocommit': True,
       'authenticator': 'externalbrowser',
       'account': 'square',
       'database': f'PERSONAL_{user_name.upper()}',
       'user': f'{user_name}@squareup.com'
   }
   
)
conn = sess.open()



In [0]:
# !pip install fastparquet

In [0]:
conn.execute('use warehouse ADHOC__LARGE')
conn.execute('use database AP_CUR_FRDRISK_G')
conn.execute('use schema public')


In [0]:
def create_base_tables(conn,user_name='jobyg' 
):
    from datetime import datetime, timedelta
    rt_start_date =  (datetime.now().today()- timedelta(days=2)).strftime("%Y-%m-%d")
    first_start_date=  (datetime.now().today()- timedelta(days=60)).strftime("%Y-%m-%d")

    print('creating attempt base')
    attempt_driver_q = f'''    
create or replace temp table jobys_latest_attempts as (
select a.*, b.p2_overdue_d0_local, b.p2_due_local
from ap_cur_r_frdrisk.curated_fraud_risk_red.unified_feature_datamart_base__{user_name}_dsl3_sv a
left join AP_CUR_RISKBI_G.curated_risk_bi_green.dwm_order_loss_tagging b
on a.order_token = b.order_token
-- where (bp_is_sup = 1 and days_since_first_order_date <= 14 and a.par_region in ('GB', 'US')
where  days_since_first_order_date <= 14 and a.par_region in ('GB', 'US','AU') and a.checkpoint = 'CHECKOUT_CONFIRM'
and a.par_process_date >= '{first_start_date}'
and dedup = 1
);
'''
    conn.execute(attempt_driver_q)

    print('creating decline base')

    rt_driver_decline_q = f'''    
create or replace temp table rt_Declines as (
SELECT order_token, rule_id, par_Region, EVENT_INFO_EVENT_TIME,rule_category
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND par_process_date >=  '{rt_start_date}'
    AND par_Region in ('AU','GB', 'US')
    and rule_id in (
'au_fraud_online_new_overdue_v3_general'
,'anz_abusive_fraud_online_whitepages_network_score_2024'
,'au_order_velocity_rule_new'
,'ANZ_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'au_fraud_online_recurring_payment_seed_based_linking_rule_v2_migrated'
,'anz_abusive_fraud_online_same_merch_email_streaming_2024'
,'anz_fraud_newconsumer_overdue_v1_replacement'
,'au_online_duplicate_account_identity_type'
,'au_fraud_doordash_decl_history_velocity_rule_v2_migrated'
,'anz_abusive_fraud_online_order_velocity_2024'
,'au_fraud_online_HRM_payback_model_v3_fast'
,'AU_fraud_online_new_user_session_device_linking_eval'
,'anz_fraud_online_newconsumer_device_check_RE_v2_migrated'
,'au_fraud_online_quasi_duplicate_account_written_off'
,'au_fraud_online_card_sharing_new_consumer'
,'au_fraud_online_new_consumer_not_first_order_v2_migrated'
,'anz_fraud_online_hrm_travel'
,'anz_fraud_online_cc_mismatch_email_age_v2_migrated'
,'AU_Online_doordash_high_risk'
,'au_fraud_udf_duplicate_account_freeze'
,'AU_fraud_online_new_user_risky_card_creation_date'
,'au_fraud_online_profile_change_phone_new_user'
,'au_fraud_online_suspicious_group_acount_tier1'
,'anz_fraud_online_velocity_acct_same_merch_email_new_v2_migrated'
,'au_fraud_online_electless_collusion_trend'


--GB SUP
,'GB_fraud_sup_strategy_odv3_general'
,'eu_fraud_online_seed_based_linking_rule_v2_migrated'
,'uk_fraud_online_sup_wpp_phone_check_decline'
,'eu_order_velocity_rule_new'
,'uk_fraud_online_delphi_v3_t14_sup_non_first_order'
,'UK_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'uk_fraud_online_allconsumer_device_check_RE_v2_migrated'
,'gb_fraud_online_quasi_duplicate_account_written_off'
,'eu_fraud_online_wpp_network_score'
,'eu_fraud_online_velocity_order_cnt_same_merch_email_new_streaming_v2_migrated'
,'GB_fraud_online_new_user_session_device_linking'
,'eu_fraud_online_velocity_order_amt_same_merch_email_new_streaming_v2_migrated'
,'eu_fraud_online_duplicate_accounts_tier2_udf_decline'
,'eu_fraud_online_duplicate_accounts_tier2_decline_v2_migrated'
,'eu_fraud_online_fraud_decline_repeated_freeze'
,'GB_fraud_online_new_user_risky_card_bin'
,'gb_fraud_online_profile_change_phone_new_user'
,'gb_fraud_online_quasi_duplicate_account_decline'
,'gb_fraud_online_card_sharing_new_consumer'
,'gb_fraud_online_suspicious_group_acount_tier1'
,'gb_online_payment_reschedule'
,'eu_fraud_online_velocity_acct_same_merch_email_new_streaming_v2_migrated'
,'eu_fraud_online_duplicate_account_collection_seed'
,'eu_fraud_udf_duplicate_account'
,'eu_fraud_online_velocity_acct_same_merch_email_new_v2_migrated'
,'GB_fraud_online_new_user_risky_card_issuing_bank'
,'gb_fraud_online_velocity_new_existing_6h_K_v2_migrated'
,'GB_fraud_online_new_user_risky_email_domain'
,'GB_online_high_order_velocity_rule_v2_migrated'
,'gb_fraud_online_same_merch_velocity'
,'EU_abusive_online_emailage_rule'

--US SUP
,'US_fraud_sup_strategy_odv3_general'
,'us_fraud_online_sup_transaction_model_wpp_info_mismatch_v1'
,'us_fraud_online_seed_based_linking_rule_v2_migrated'
,'NA_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'us_fraud_online_whitepages_general_decline'
,'us_fraud_udf_income_zipcode'
,'cash_credit_abuse_model_rule_v3'
,'us_sup_order_velocity_rule_new'
,'us_fraud_udf_duplicate_account'
,'US_fraud_online_new_user_session_device_linking'
,'us_fraud_online_quasi_duplicate_account_written_off'
,'US_fraud_online_new_user_risky_card_issuing_bank'
,'us_fraud_online_duplicate_accounts_tier2_udf_decline'
,'us_fraud_online_duplicate_accounts_tier2_decline_v2_migrated'
,'cash_credit_abuse_model_rule_realtime_v2'
,'us_fraud_online_fraud_decline_repeated_freeze'
,'us_fraud_online_profile_change_fraud_decline'
,'us_fraud_online_card_sharing_new_consumer'
,'us_fraud_online_velocity_same_merch_email_decline'
,'us_fraud_online_profile_change_phone_new_user'
,'us_fraud_udf_duplicate_account_freeze'
,'us_fraud_online_duplicate_account_collection_seed'
,'US_fraud_online_new_user_risky_card_bin'
,'us_fraud_online_quasi_duplicate_account_decline'
,'US_online_newuser_order_velocity_decline'
,'us_fraud_online_suspicious_group_acount_v2_migrated'
,'US_fraud_online_new_user_risky_email_domain'
,'us_fraud_online_suspicious_group_acount_tier3_v2_migrated'
,'us_fraud_online_velocity_acct_same_merch_email_new_streaming_v2_migrated'
,'us_fraud_online_payment_reschedule'
,'us_fraud_online_new_risky_card_issuing_bank'
,'us_fraud_online_suspicious_group_acount_tier2_v2_migrated'
,'us_fraud_online_duplicate_gmail_accts_new'
,'us_fraud_sup_strategy_whitepages_general_v2'
,'us_fraud_online_velocity_acct_same_merch_email_new_V2'

-- US ONLINE

,'NA_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'US_abusive_online_emailage_rule'
));
'''
    conn.execute(rt_driver_decline_q)

    old_driver_decline_q = f'''create or replace temp table old_declines as (
SELECT order_token, rule_id, par_Region, EVENT_INFO_EVENT_TIME, rule_category
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.tBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND par_Region in ('AU','GB', 'US')
    and rule_id in (
'au_fraud_online_new_overdue_v3_general'
,'anz_abusive_fraud_online_whitepages_network_score_2024'
,'au_order_velocity_rule_new'
,'ANZ_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'au_fraud_online_recurring_payment_seed_based_linking_rule_v2_migrated'
,'anz_abusive_fraud_online_same_merch_email_streaming_2024'
,'anz_fraud_newconsumer_overdue_v1_replacement'
,'au_online_duplicate_account_identity_type'
,'au_fraud_doordash_decl_history_velocity_rule_v2_migrated'
,'anz_abusive_fraud_online_order_velocity_2024'
,'au_fraud_online_HRM_payback_model_v3_fast'
,'AU_fraud_online_new_user_session_device_linking_eval'
,'anz_fraud_online_newconsumer_device_check_RE_v2_migrated'
,'au_fraud_online_quasi_duplicate_account_written_off'
,'au_fraud_online_card_sharing_new_consumer'
,'au_fraud_online_new_consumer_not_first_order_v2_migrated'
,'anz_fraud_online_hrm_travel'
,'anz_fraud_online_cc_mismatch_email_age_v2_migrated'
,'AU_Online_doordash_high_risk'
,'au_fraud_udf_duplicate_account_freeze'
,'AU_fraud_online_new_user_risky_card_creation_date'
,'au_fraud_online_profile_change_phone_new_user'
,'au_fraud_online_suspicious_group_acount_tier1'
,'anz_fraud_online_velocity_acct_same_merch_email_new_v2_migrated'
,'au_fraud_online_electless_collusion_trend'

--GB SUP
,'GB_fraud_sup_strategy_odv3_general'
,'eu_fraud_online_seed_based_linking_rule_v2_migrated'
,'uk_fraud_online_sup_wpp_phone_check_decline'
,'eu_order_velocity_rule_new'
,'uk_fraud_online_delphi_v3_t14_sup_non_first_order'
,'UK_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'uk_fraud_online_allconsumer_device_check_RE_v2_migrated'
,'gb_fraud_online_quasi_duplicate_account_written_off'
,'eu_fraud_online_wpp_network_score'
,'eu_fraud_online_velocity_order_cnt_same_merch_email_new_streaming_v2_migrated'
,'GB_fraud_online_new_user_session_device_linking'
,'eu_fraud_online_velocity_order_amt_same_merch_email_new_streaming_v2_migrated'
,'eu_fraud_online_duplicate_accounts_tier2_udf_decline'
,'eu_fraud_online_duplicate_accounts_tier2_decline_v2_migrated'
,'eu_fraud_online_fraud_decline_repeated_freeze'
,'GB_fraud_online_new_user_risky_card_bin'
,'gb_fraud_online_profile_change_phone_new_user'
,'gb_fraud_online_quasi_duplicate_account_decline'
,'gb_fraud_online_card_sharing_new_consumer'
,'gb_fraud_online_suspicious_group_acount_tier1'
,'gb_online_payment_reschedule'
,'eu_fraud_online_velocity_acct_same_merch_email_new_streaming_v2_migrated'
,'eu_fraud_online_duplicate_account_collection_seed'
,'eu_fraud_udf_duplicate_account'
,'eu_fraud_online_velocity_acct_same_merch_email_new_v2_migrated'
,'GB_fraud_online_new_user_risky_card_issuing_bank'
,'gb_fraud_online_velocity_new_existing_6h_K_v2_migrated'
,'GB_fraud_online_new_user_risky_email_domain'
,'GB_online_high_order_velocity_rule_v2_migrated'
,'gb_fraud_online_same_merch_velocity'
,'EU_abusive_online_emailage_rule'

--US SUP
,'US_fraud_sup_strategy_odv3_general'
,'us_fraud_online_sup_transaction_model_wpp_info_mismatch_v1'
,'us_fraud_online_seed_based_linking_rule_v2_migrated'
,'NA_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'us_fraud_online_whitepages_general_decline'
,'us_fraud_udf_income_zipcode'
,'cash_credit_abuse_model_rule_v3'
,'us_sup_order_velocity_rule_new'
,'us_fraud_udf_duplicate_account'
,'US_fraud_online_new_user_session_device_linking'
,'us_fraud_online_quasi_duplicate_account_written_off'
,'US_fraud_online_new_user_risky_card_issuing_bank'
,'us_fraud_online_duplicate_accounts_tier2_udf_decline'
,'us_fraud_online_duplicate_accounts_tier2_decline_v2_migrated'
,'cash_credit_abuse_model_rule_realtime_v2'
,'us_fraud_online_fraud_decline_repeated_freeze'
,'us_fraud_online_profile_change_fraud_decline'
,'us_fraud_online_card_sharing_new_consumer'
,'us_fraud_online_velocity_same_merch_email_decline'
,'us_fraud_online_profile_change_phone_new_user'
,'us_fraud_udf_duplicate_account_freeze'
,'us_fraud_online_duplicate_account_collection_seed'
,'US_fraud_online_new_user_risky_card_bin'
,'us_fraud_online_quasi_duplicate_account_decline'
,'US_online_newuser_order_velocity_decline'
,'us_fraud_online_suspicious_group_acount_v2_migrated'
,'US_fraud_online_new_user_risky_email_domain'
,'us_fraud_online_suspicious_group_acount_tier3_v2_migrated'
,'us_fraud_online_velocity_acct_same_merch_email_new_streaming_v2_migrated'
,'us_fraud_online_payment_reschedule'
,'us_fraud_online_new_risky_card_issuing_bank'
,'us_fraud_online_suspicious_group_acount_tier2_v2_migrated'
,'us_fraud_online_duplicate_gmail_accts_new'
,'us_fraud_sup_strategy_whitepages_general_v2'
,'us_fraud_online_velocity_acct_same_merch_email_new_V2'

--US ONLINE
,'NA_FRAUD_TMX_ONLINE_NEW_USERS_V1'
,'US_abusive_online_emailage_rule'
)

    AND par_process_date >=  '{first_start_date}'
    and par_process_date <= '{rt_start_date}');'''

    conn.execute(old_driver_decline_q)

    conn.execute('''create or replace temp table full_declines as (
select * from rt_declines union select * from old_Declines
);''')
    
    print('working on unique declines')

    conn.execute(f'''create or replace temp table unique_declines_rt as (
SELECT order_token, count(distinct(rule_id)) as rule_ct, 
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND par_Region in ('GB','US','AU')
    AND par_process_date >=  '{first_start_date}'
    and par_process_date <= '{rt_start_date}'
    group by 1
    having  count(distinct(rule_id)) = 1
);''')
    
    conn.execute(f'''
                 create or replace temp table unique_declines_old as (
SELECT order_token, count(distinct(rule_id)) as rule_ct, 
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND par_process_date >=  '{first_start_date}'
    and par_process_date <= '{rt_start_date}'
    AND par_Region in ('GB','US','AU')
    group by 1
    having  count(distinct(rule_id)) = 1
); ''')
    
    conn.execute('''create or replace temp table unique_declines as (
select * from unique_declines_rt union select * from unique_declines_old
);''')
    
    print('working on trust')
    conn.execute(f'''create or replace temp table trust_rt as (
SELECT order_token, count(distinct(rule_id)) as rule_ct, 
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND actions ilike '%trust%'
    AND par_Region in ('GB','US','AU')
    AND par_process_date >=  '{first_start_date}'
    and par_process_date <= '{rt_start_date}'
    group by 1
);''')
    conn.execute(f'''create or replace temp table trust_old as (
SELECT order_token, count(distinct(rule_id)) as rule_ct, 
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    where checkpoint in ('CHECKOUT_CONFIRM')
    AND actions ilike '%trust%'
    AND par_process_date >=  '{first_start_date}'
    and par_process_date <= '{rt_start_date}'
    AND par_region in ('GB','US','AU')
    group by 1
);''')

    conn.execute('''create or replace temp table trust as (
select * from trust_rt union select * from trust_old
);''')

import pandas as pd

def summarize_and_compare_metrics_v2(df, country, bp_is_sup, p2d0_lag_days=15):
    df = df[(df['bp_is_sup'] == bp_is_sup) & (df['par_region'] == f'{country}')]
    df['par_process_date'] = pd.to_datetime(df['par_process_date'])
    latest_date = df['par_process_date'].max()

    def get_week_bounds(reference_date):
        end = reference_date - pd.Timedelta(days=reference_date.weekday() - 6)
        start = end - pd.Timedelta(days=6)
        prev_end = start - pd.Timedelta(days=1)
        prev_start = prev_end - pd.Timedelta(days=6)
        return (start, end, prev_start, prev_end)

    def pct_change(current, previous):
        if pd.isna(current) or pd.isna(previous) or previous == 0:
            return None
        return (current - previous) / previous * 100

    def format_pct(value):
        return f"{value:.1f}%" if value is not None else "N/A"

    def format_val(value, is_pct=False):
        if value is None:
            return "N/A"
        return f"{value:.2%}" if is_pct else f"${value:,.0f}"

    def weighted_avg(df_subset):
        attempts = df_subset['attempt_ct'].sum()
        return (df_subset['decline_rt'] * df_subset['attempt_ct']).sum() / attempts if attempts else None

    ### 1. Decline Rate
    decline_ref = latest_date
    cur_start, cur_end, prev_start, prev_end = get_week_bounds(decline_ref)

    df_today = df[df['par_process_date'] == decline_ref]
    df_yesterday = df[df['par_process_date'] == decline_ref - pd.Timedelta(days=1)]
    df_wow = df[(df['par_process_date'] >= cur_start) & (df['par_process_date'] <= cur_end)]
    df_prev_wow = df[(df['par_process_date'] >= prev_start) & (df['par_process_date'] <= prev_end)]

    decline_today = weighted_avg(df_today)
    decline_yesterday = weighted_avg(df_yesterday)
    decline_wow = weighted_avg(df_wow)
    decline_prev_wow = weighted_avg(df_prev_wow)

    ### 2. Approved Amount
    approved_ref = latest_date - pd.Timedelta(days=2)
    cur_start, cur_end, prev_start, prev_end = get_week_bounds(approved_ref)

    approved_today = df[df['par_process_date'] == approved_ref]['approved_amt'].sum()
    approved_yesterday = df[df['par_process_date'] == approved_ref - pd.Timedelta(days=1)]['approved_amt'].sum()
    # print(cur_start,cur_end)
    approved_wow = df[(df['par_process_date'] >= cur_start) & (df['par_process_date'] <= cur_end)]['approved_amt'].sum()
    approved_prev_wow = df[(df['par_process_date'] >= prev_start) & (df['par_process_date'] <= prev_end)]['approved_amt'].sum()

    ### 3. P2D0 Rate
    p2d0_ref = latest_date - pd.Timedelta(days=p2d0_lag_days)
    cur_start, cur_end, prev_start, prev_end = get_week_bounds(p2d0_ref)

    p2d0_today = df[df['par_process_date'] == p2d0_ref]['portfolio_p2_do'].mean()
    p2d0_yesterday = df[df['par_process_date'] == p2d0_ref - pd.Timedelta(days=1)]['portfolio_p2_do'].mean()
    p2d0_wow = df[(df['par_process_date'] >= cur_start) & (df['par_process_date'] <= cur_end)]['portfolio_p2_do'].mean()
    p2d0_prev_wow = df[(df['par_process_date'] >= prev_start) & (df['par_process_date'] <= prev_end)]['portfolio_p2_do'].mean()

    ### Final summary
    lines = []

    # Decline rate summary
    dod_decline = pct_change(decline_today, decline_yesterday)
    wow_decline = pct_change(decline_wow, decline_prev_wow)
    if bp_is_sup:
        lines.append(f'performance for {country} SUP is:')
    else:
        lines.append(f'performance for {country} Online is:')


    lines.append(f"📉 *Decline Rate* (as of {decline_ref.date()}): {format_val(decline_today, is_pct=True)} "
                 f"(DoD: {format_pct(dod_decline)}, WoW: {format_pct(wow_decline)})")

    # Approved amount summary
    dod_approved = pct_change(approved_today, approved_yesterday)
    wow_approved = pct_change(approved_wow, approved_prev_wow)
    lines.append(f"💰 *Approved Amount* (as of {approved_ref.date()}): {format_val(approved_today)} "
                 f"(DoD: {format_pct(dod_approved)}, WoW: {format_pct(wow_approved)})")

    # P2D0 rate summary
    dod_p2d0 = pct_change(p2d0_today, p2d0_yesterday)
    wow_p2d0 = pct_change(p2d0_wow, p2d0_prev_wow)
    lines.append(f"⏱️ *P2D0 Rate* (as of {p2d0_ref.date()}): {format_val(p2d0_today, is_pct=True)} "
                 f"(DoD: {format_pct(dod_p2d0)}, WoW: {format_pct(wow_p2d0)})")

    output = '\n'.join(lines)
    print(output)    

def create_graphs(df, metric,bp_is_sup):

    import seaborn as sns
    import matplotlib.pyplot as plt

# --- Step 1: Filter by bp_is_sup values ---
    df_sup = df[df['bp_is_sup'] == bp_is_sup]

# --- Step 2: Group and aggregate the data by date and region ---
    df_sup_agg = df_sup.groupby(['par_process_date', 'par_region'])[metric].mean().reset_index()

# --- Step 3: Plot for bp_is_sup = 1 ---
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=df_sup_agg,
        x='par_process_date',
        y=metric,
        hue='par_region',
        marker='o'
)
    plt.title(f'{metric} Over Time by Region (bp_is_sup = {bp_is_sup})')
    plt.xlabel('Processing Date')
    plt.ylabel(f'{metric}')
    plt.legend(title='Region')
    plt.ylim(0, 1)

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


import pandas as pd
from datetime import timedelta

def compare_wtd_metrics_from_hourly_df(df, now=None, region_filter=None, bp_is_sup = None):
    # Parse checkout_time
    df = df.copy()
    df['checkout_time'] = pd.to_datetime(df['checkout_time'], format='%Y-%m-%d %H')
    df = df[df['bp_is_sup'] == bp_is_sup]
    if region_filter:
        df = df[df['par_region'] == f'{region_filter}']

    now = pd.Timestamp.now() if now is None else pd.to_datetime(now)

    def get_wtd_window(now_shifted):
        weekday = now_shifted.weekday()
        latest_sunday = now_shifted - pd.Timedelta(days=weekday + 1)
        start = pd.Timestamp.combine(latest_sunday.date(), pd.Timestamp.min.time())
        end = now_shifted.floor('H')
        prev_start = start - pd.Timedelta(days=7)
        prev_end = end - pd.Timedelta(days=7)
        return start, end, prev_start, prev_end

    def pct_change(curr, prev):
        if curr is None or prev is None or prev == 0:
            return None
        return (curr - prev) / prev * 100

    def format_pct(x): return f"{x:.1f}%" if x is not None else "N/A"
    def format_val(x, pct=False): return f"{x:.2%}" if pct and x is not None else f"${x:,.0f}" if x is not None else "N/A"

    def summarize(sub_df):
        attempts = sub_df['attempt_ct'].sum()
        declines = sub_df['decline_ct'].sum()
        approved = sub_df['approved_amt'].sum()
        p2_overdue = sub_df['p2_overdue'].sum()
        p2_due = sub_df['p2_due_local'].sum()

        decline_rt = declines / attempts if attempts else None
        p2d0_rt = p2_overdue / p2_due if p2_due else None

        return {
            'attempts': attempts,
            'decline_rate': decline_rt,
            'approved_amt': approved,
            'p2d0_rate': p2d0_rt
        }

    lines = []
    if bp_is_sup:
        lines.append(f'performance for {region_filter} SUP is:')
    else:
        lines.append(f'performance for {region_filter} Online is:')
    ### 1. Decline Rate — real-time
    d_start, d_end, d_prev_start, d_prev_end = get_wtd_window(now)
    cur_d = df[(df['checkout_time'] >= d_start) & (df['checkout_time'] <= d_end)]
    prev_d = df[(df['checkout_time'] >= d_prev_start) & (df['checkout_time'] <= d_prev_end)]

    d_now = summarize(cur_d)
    d_prev = summarize(prev_d)
    d_delta = pct_change(d_now['decline_rate'], d_prev['decline_rate'])
    lines.append(f"📉 *Decline Rate* (WTD up to {d_end:%Y-%m-%d %H:%M}): "
                 f"{format_val(d_now['decline_rate'], pct=True)} (vs last week: {format_pct(d_delta)})")

    ### 2. Approved Amount — 1-day lag
    a_now_time = now - timedelta(days=1)
    a_start, a_end, a_prev_start, a_prev_end = get_wtd_window(a_now_time)
    cur_a = df[(df['checkout_time'] >= a_start) & (df['checkout_time'] <= a_end)]
    prev_a = df[(df['checkout_time'] >= a_prev_start) & (df['checkout_time'] <= a_prev_end)]

    a_now = summarize(cur_a)
    a_prev = summarize(prev_a)
    a_delta = pct_change(a_now['approved_amt'], a_prev['approved_amt'])
    lines.append(f"💰 *Approved Amount* (WTD ending {a_end:%Y-%m-%d}): "
                 f"{format_val(a_now['approved_amt'])} (vs last week: {format_pct(a_delta)})")

    ### 3. P2D0 Rate — 14-day lag
    p_now_time = now - timedelta(days=14)
    p_start, p_end, p_prev_start, p_prev_end = get_wtd_window(p_now_time)
    cur_p = df[(df['checkout_time'] >= p_start) & (df['checkout_time'] <= p_end)]
    prev_p = df[(df['checkout_time'] >= p_prev_start) & (df['checkout_time'] <= p_prev_end)]

    p_now = summarize(cur_p)
    p_prev = summarize(prev_p)
    p_delta = pct_change(p_now['p2d0_rate'], p_prev['p2d0_rate'])
    lines.append(f"⏱️ *P2D0 Rate* (WTD ending {p_end:%Y-%m-%d}): "
                 f"{format_val(p_now['p2d0_rate'], pct=True)} (vs last week: {format_pct(p_delta)})")

    header = f"📊 *Adjusted Week-to-Date Comparison* (as of {now:%A %Y-%m-%d %H:%M})"
    output = '\n'.join(lines)
    print(output)
import pandas as pd

def generate_rule_change_report(df, focus_region='US', top_n=10, bp_is_sup = None, ref_date=None, min_decline_rt = .01, conn=conn):
    df['par_process_date'] = pd.to_datetime(df['par_process_date'])
    ref_date = pd.to_datetime(ref_date) if ref_date else df['par_process_date'].max()

    # Filter for current region and baseline day
    df = df[(df['par_region'] == f'{focus_region}') & (df['bp_is_sup'] == bp_is_sup)]

    # Define historical reference dates
     # Reference date mapping
    dates = {
        'today': ref_date,
        'yesterday': ref_date - pd.Timedelta(days=1),
        'last_week': ref_date - pd.Timedelta(days=7),
        'last_month': ref_date - pd.Timedelta(days=30)
    }

    # Keep only those dates
    df_compare = df[df['par_process_date'].isin(dates.values())]
    metrics = ['decline_ct', 'attempt_ct', 'decline_rt']

    rule_summary = (
        df_compare
        .groupby(['rule_id', 'par_process_date'])[metrics]
        .sum()
        .reset_index()
    )

    pivot = rule_summary.pivot(index='rule_id', columns='par_process_date', values=metrics)
    pivot.columns = [f"{m}_{d.strftime('%Y-%m-%d')}" for m, d in pivot.columns]
    pivot = pivot.reset_index()

    # Helper functions
    def pct_change(new, old):
        if pd.isna(new) or pd.isna(old) or old == 0:
            return None
        return (new - old) / old * 100

    def fmt(val, pct=False):
        if pd.isna(val):
            return "N/A"
        return f"{val:.2%}" if pct else f"{val:,.0f}"

    def fmt_pct(val): return f"{val:+.1f}%" if val is not None else "N/A"

    # Date suffixes
    t = dates['today'].strftime('%Y-%m-%d')
    y = dates['yesterday'].strftime('%Y-%m-%d')
    w = dates['last_week'].strftime('%Y-%m-%d')
    m = dates['last_month'].strftime('%Y-%m-%d')

    # Calculate deltas
    pivot['delta_decline_rt_y'] = pivot.apply(lambda r: pct_change(r.get(f'decline_rt_{t}'), r.get(f'decline_rt_{y}')), axis=1)
    pivot['delta_decline_rt_w'] = pivot.apply(lambda r: pct_change(r.get(f'decline_rt_{t}'), r.get(f'decline_rt_{w}')), axis=1)
    pivot['delta_decline_rt_m'] = pivot.apply(lambda r: pct_change(r.get(f'decline_rt_{t}'), r.get(f'decline_rt_{m}')), axis=1)
    pivot['delta_decline_ct'] = pivot.get(f'decline_ct_{t}', 0) - pivot.get(f'decline_ct_{y}', 0)

    # Absolute values for sorting
    pivot['abs_rt_change_y'] = pivot['delta_decline_rt_y'].abs()
    pivot['abs_rt_change_w'] = pivot['delta_decline_rt_w'].abs()
    pivot['abs_rt_change_m'] = pivot['delta_decline_rt_m'].abs()

    # Apply threshold filter
    pivot = pivot[pivot[f'decline_rt_{t}'] > min_decline_rt]

    # Top N rules by absolute daily decline rate change
    top_rules = pivot.sort_values(by=['abs_rt_change_y', 'abs_rt_change_w', 'abs_rt_change_m'], ascending=False).head(top_n)

    # Assemble report
    lines = [f"📊 *Top {top_n} Rule Decline Rate Movers for {focus_region} on {t} (Filtered: decline rate > {min_decline_rt:.0%})*"]

    for _, row in top_rules.iterrows():
        rid = row['rule_id']
        attempts = row.get(f'attempt_ct_{t}', 0)
        declines = row.get(f'decline_ct_{t}', 0)
        drt = row.get(f'decline_rt_{t}', None)

        lines.append(
            f"\n🔹 Rule `{rid}`: {int(attempts)} attempts, {int(declines)} declines"
            f"\n  - Decline Rate: {fmt(drt, pct=True)}"
            f"\n  - vs Yesterday: {fmt_pct(row['delta_decline_rt_y'])}"
            f"\n  - vs Last Week: {fmt_pct(row['delta_decline_rt_w'])}"
            f"\n  - vs Last Month: {fmt_pct(row['delta_decline_rt_m'])}"
        )   
    output= "\n".join(lines)
    print(output)

def create_stacked_100_graph(df, metric='gmv', top_n=15):
    """
    Plots a 100% stacked column chart of a given metric by merchant per day.
    
    Parameters:
        df (DataFrame): Data with columns ['par_process_date', 'in_flight_order_merchant_name', metric]
        metric (str): 'gmv' or 'p2_overdue'
        top_n (int): Number of merchants to include per day
    """
    df = df.copy()
    df['par_process_date'] = pd.to_datetime(df['par_process_date'])
    df = df.rename(columns={'in_flight_order_merchant_name': 'merchant'})

    # Filter top N merchants per day
    df['_rank'] = df.groupby('par_process_date')[metric].rank(method='first', ascending=False)
    df_top = df[df['_rank'] <= top_n].copy()
    df_top.drop(columns=['_rank'], inplace=True)

    # Pivot to merchant x date table
    pivot = df_top.pivot_table(
        index='par_process_date',
        columns='merchant',
        values=metric,
        aggfunc='sum',
        fill_value=0
    )

    # Normalize rows to 100%
    pct = pivot.div(pivot.sum(axis=1), axis=0)

    # Plot
    fig, ax = plt.subplots(figsize=(14, 6))
    bottom = pd.Series(0, index=pct.index)
    colors = plt.cm.tab20.colors  # pick from matplotlib's default categorical palette

    for i, merchant in enumerate(pct.columns):
        ax.bar(pct.index, pct[merchant], bottom=bottom, label=merchant, color=colors[i % len(colors)])
        bottom += pct[merchant]

    ax.set_title(f'100% Stacked Column Chart of {metric.upper()} by Merchant')
    ax.set_ylabel('Percentage')
    ax.set_xlabel('Date')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Merchant')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def flag_disproportionate_p2_overdue(df, threshold_ratio=1.5, min_p2d0_rate=0.25, min_p2_overdue_amt=200):
    """
    Identifies merchants whose P2 overdue contributions are disproportionately high compared to their GMV.

    Parameters:
        df (DataFrame): From the US_merchant_analysis query. Must contain:
                        ['par_process_date', 'in_flight_order_merchant_name', 'gmv', 'p2_overdue', 'p2d0_rate']
        threshold_ratio (float): How much larger p2_overdue share must be vs gmv share to flag.
        min_p2d0_rate (float): Minimum P2D0 rate threshold to consider a merchant 'problematic'.

    Returns:
        DataFrame: Merchants flagged as disproportionately contributing to P2 overdue.
    """
    df = df.copy()
    df['par_process_date'] = pd.to_datetime(df['par_process_date'])
    df = df.rename(columns={'in_flight_order_merchant_name': 'merchant'})

    # Compute total GMV and total P2 overdue per day
    totals = df.groupby('par_process_date').agg({
        'gmv': 'sum',
        'p2_overdue': 'sum'
    }).rename(columns={'gmv': 'total_gmv', 'p2_overdue': 'total_p2_overdue'})

    # Join totals back to original df
    df = df.merge(totals, on='par_process_date', how='left')

    # Calculate shares and flag
    df['gmv_share'] = df['gmv'] / df['total_gmv']
    df['p2_overdue_share'] = df['p2_overdue'] / df['total_p2_overdue']
    df['share_ratio'] = df['p2_overdue_share'] / df['gmv_share']
    df['flag_disproportionate'] = (df['share_ratio'] >= threshold_ratio) & (df['p2d0_rate'] >= min_p2d0_rate) & (df['p2_overdue'] >= min_p2_overdue_amt)

    # Return flagged merchants only
    flagged_df = df[df['flag_disproportionate']].sort_values(['par_process_date', 'share_ratio'], ascending=[False, False])
    return flagged_df
    

In [0]:
create_base_tables(user_name=user_name,conn=conn) 

In [0]:
# from pysnowflake import Session
# df = conn.download('select * from jobys_latest_attempts limit 5')


In [0]:
df = conn.download('''
                                  select
a.par_Region,
par_process_date
,bp_is_sup
,count(distinct(a.order_token)) as attempt_ct
,count(distinct(a.order_token_loss)) as approved_order_Ct
,sum(in_flight_order_amount) as attempt_Amt
,sum(order_amount_local) as approved_amt
,count(distinct b.order_token) as decline_Ct
,div0(count(distinct b.order_token),count(distinct(a.order_token))) as decline_rt
,div0(count(distinct order_Token_loss),count(distinct(a.order_token))) as decline_including_Trust
,div0(sum(p2_overdue_d0_local),sum(p2_due_local)) as approved_p2d0
,div0(sum(case when is_in_attempt_control_Group = 1 then p2_overdue_d0_local end),sum(case when is_in_attempt_control_Group = 1 then p2_due_local end)) as ctrl_Group_p2d0
,div0(sum(p2_overdue_d0_local),sum( p2_due_local)) as portfolio_p2_do

,sum(case when b.order_Token is not null then in_flight_order_amount end) as decline_amt
from jobys_latest_attempts a
left join full_declines b
on a.order_token = b.order_token
where a.par_process_date >=  DATEADD('day', -60, CURRENT_DATE())
-- and a.par_region = 'US'
-- and bp_is_sup = 1
group by 1,2,3
order by 1,2,3 desc;
''')

In [0]:
df.columns
# import matplotlib.pyplot as plt


In [0]:
create_graphs(df, 'decline_rt',1)

In [0]:
create_graphs(df, 'decline_rt',0)

In [0]:
create_graphs(df, 'portfolio_p2_do',0)

In [0]:
create_graphs(df, 'portfolio_p2_do',1)

In [0]:
summarize_and_compare_metrics_v2(df, 'US', 1)

In [0]:
summarize_and_compare_metrics_v2(df, 'GB', 1)

In [0]:
summarize_and_compare_metrics_v2(df, 'US', 0)

In [0]:
summarize_and_compare_metrics_v2(df, 'AU', 0)

In [0]:
week_to_date_df = conn.download('''
select
a.par_Region,
to_Varchar(checkout_time,'YYYY-MM-DD HH') as checkout_time
,bp_is_sup
,count(distinct(a.order_token)) as attempt_ct
,sum(order_amount_local) as approved_amt
,count(distinct b.order_token) as decline_ct
,sum(p2_overdue_d0_local) as p2_overdue
,sum(p2_due_local) as p2_due_local
from jobys_latest_attempts a
left join full_declines b
on a.order_token = b.order_token
where a.par_process_date >=  DATEADD('day', -30, CURRENT_DATE())
-- and a.par_region = 'US'
-- and bp_is_sup = 1
group by 1,2,3
order by 1,2,3 desc;
''')

In [0]:
compare_wtd_metrics_from_hourly_df(week_to_date_df, region_filter = 'US', bp_is_sup = 1)

In [0]:
compare_wtd_metrics_from_hourly_df(week_to_date_df, region_filter = 'GB', bp_is_sup = 1)

In [0]:
compare_wtd_metrics_from_hourly_df(week_to_date_df, region_filter = 'AU', bp_is_sup = 0)

In [0]:
compare_wtd_metrics_from_hourly_df(week_to_date_df, region_filter = 'US', bp_is_sup = 0)

In [0]:
compare_wtd_metrics_from_hourly_df(week_to_date_df, region_filter = 'GB', bp_is_sup = 0)

In [0]:
# focus_region='US'
bp_is_sup=1
conn.execute(f'''create or replace temp table decline_rt_denom as (
 select par_process_date, par_region, bp_is_sup, count(distinct(order_Token)) as token_ct from jobys_latest_attempts 
     group by 1,2,3
    );''')

rule_daily_dive = conn.download('''select
a.par_Region,
a.par_process_date
,a.bp_is_sup
,rule_id
,count(distinct(a.order_token)) as decline_ct
,count(distinct(a.order_token_loss)) as approved_order_Ct
,sum(in_flight_order_amount) as attempt_Amt
,sum(order_amount_local) as approved_amt
,max(c.token_ct) as attempt_ct
,count(distinct a.order_token)/max(c.token_ct) as decline_rt
,div0(sum(case when is_in_attempt_control_Group = 1 then p2_overdue_d0_local end),sum(case when is_in_attempt_control_Group = 1 then p2_due_local end)) as ctrl_Group_p2d0
,div0(sum(p2_overdue_d0_local),sum( p2_due_local)) as portfolio_p2_do

,sum(case when b.order_Token is not null then in_flight_order_amount end) as decline_amt
from jobys_latest_attempts a
left join (select distinct order_token, rule_id from full_declines where rule_Category not in ('SF', 'ATO', 'UNAUTH') ) b
on a.order_token = b.order_token
left join decline_rt_denom c
on a.par_region = c.par_Region
and a.bp_is_sup = c.bp_is_sup
and a.par_process_date = c.par_process_date
where a.par_process_date >=  DATEADD('day', -33, CURRENT_DATE())

group by 1,2,3,4
order by 1,2,3,4 desc;''')
    

In [0]:
# # 
# rule_daily_dive.loc[(rule_daily_dive.rule_id == 'UK_FRAUD_TMX_ONLINE_NEW_USERS_V1') & (rule_daily_dive.bp_is_sup==0)].sort_values(by='par_process_date')

In [0]:
generate_rule_change_report(rule_daily_dive, focus_region='US', top_n=10, bp_is_sup = 1)

In [0]:
generate_rule_change_report(rule_daily_dive, focus_region='GB', top_n=10, bp_is_sup = 1)

In [0]:
generate_rule_change_report(rule_daily_dive, focus_region='AU', top_n=10, bp_is_sup = 0)

In [0]:
generate_rule_change_report(rule_daily_dive, focus_region='US', top_n=10, bp_is_sup = 0)

In [0]:
generate_rule_change_report(rule_daily_dive, focus_region='GB', top_n=10, bp_is_sup = 0)

In [0]:
US_merchant_analysis = conn.download('''
SELECT 
  par_process_date, 
  in_flight_order_merchant_name, 
  SUM(order_amount_local) AS gmv, 
  SUM(p2_due_local) AS p2_due_local,
  SUM(p2_overdue_d0_local) AS p2_overdue,
  DIV0(SUM(p2_overdue_d0_local), SUM(p2_due_local)) AS p2d0_rate,
 ROW_NUMBER() OVER (
    PARTITION BY par_process_date
    ORDER BY SUM(order_amount_local) DESC) as rn
FROM jobys_latest_attempts
WHERE bp_is_sup = 1
  AND par_region = 'US'
GROUP BY par_process_date, in_flight_order_merchant_name
having SUM(order_amount_local) > 0
qualify rn <= 15
''')

In [0]:
# US_merchant_analysis

In [0]:
create_stacked_100_graph(US_merchant_analysis, 'gmv', top_n=8)

In [0]:
create_stacked_100_graph(US_merchant_analysis, 'p2_overdue', top_n=8)

In [0]:
!pip install ace_tools

In [0]:
flag_disproportionate_p2_overdue(US_merchant_analysis, threshold_ratio = 2)

In [0]:
 df[(df['bp_is_sup'] == 0) & (df['par_region']=='US')].sort_values(by='par_process_date')[['par_region','par_process_date','attempt_ct','approved_amt','decline_rt','portfolio_p2_do']].tail(20)


In [0]:
df[(df['bp_is_sup'] == 0) & (df['par_region']=='AU')].sort_values(by='par_process_date')[['par_region','par_process_date','attempt_ct','approved_amt','decline_rt','portfolio_p2_do']]


In [0]:
df[(df['bp_is_sup'] == 1) & (df['par_region']=='US')].sort_values(by='par_process_date')[['par_region','par_process_date','attempt_ct','approved_amt','decline_rt','portfolio_p2_do']]


In [0]:
df[(df['bp_is_sup'] == 1) & (df['par_region']=='US')].sort_values(by='par_process_date')[['par_region','par_process_date','attempt_ct','approved_amt','decline_rt','portfolio_p2_do']]


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt


# --- Step 3: Plot for bp_is_sup = 1 ---
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_sup_0_agg,
    x='par_process_date',
    y='decline_rt',
    hue='par_region',
    marker='o'
)
plt.title('Decline Rate Over Time by Region (bp_is_sup = 0)')
plt.xlabel('Processing Date')
plt.ylabel('Decline Rate')
plt.legend(title='Region')
plt.ylim(0, 1)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

# --- Step 1: Filter by bp_is_sup values ---
df_sup_1 = df[df['bp_is_sup'] == 1]
df_sup_0 = df[df['bp_is_sup'] == 0]

# --- Step 2: Group and aggregate the data by date and region ---
df_sup_1_agg = df_sup_1.groupby(['par_process_date', 'par_region'])['portfolio_p2_do'].mean().reset_index()
df_sup_0_agg = df_sup_0.groupby(['par_process_date', 'par_region'])['portfolio_p2_do'].mean().reset_index()

# --- Step 3: Plot for bp_is_sup = 1 ---
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_sup_1_agg,
    x='par_process_date',
    y='portfolio_p2_do',
    hue='par_region',
    marker='o'
)
plt.title('Decline Rate Over Time by Region (bp_is_sup = 1)')
plt.xlabel('Processing Date')
plt.ylabel('portfolio_p2_do Rate')
plt.legend(title='Region')
plt.ylim(0, 1)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
df_sup_0

In [0]:
# --- Step 2: Group and aggregate the data by date and region ---
df_sup_1_agg = df_sup_1.groupby(['par_process_date', 'par_region'])['portfolio_p2_do'].mean().reset_index()
df_sup_0_agg = df_sup_0.groupby(['par_process_date', 'par_region'])['portfolio_p2_do'].mean().reset_index()

# --- Step 3: Plot for bp_is_sup = 1 ---
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_sup_1_agg,
    x='par_process_date',
    y='portfolio_p2_do',
    hue='par_region',
    marker='o'
)
plt.title('Decline Rate Over Time by Region (bp_is_sup = 1)')
plt.xlabel('Processing Date')
plt.ylabel('portfolio_p2_do Rate')
plt.legend(title='Region')
plt.ylim(0, 1)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_sup_0_agg,
    x='par_process_date',
    y='portfolio_p2_do',
    hue='par_region',
    marker='o'
)
plt.title('P2D0_RATE by Region (bp_is_sup = 0)')
plt.xlabel('Processing Date')
plt.ylabel('portfolio_p2_do Rate')
plt.legend(title='Region')
plt.ylim(0, .5)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_sup_1_agg,
    x='par_process_date',
    y='portfolio_p2_do',
    hue='par_region',
    marker='o'
)
plt.title('P2D0_RATE by Region (bp_is_sup = 1)')
plt.xlabel('Processing Date')
plt.ylabel('portfolio_p2_do Rate')
plt.legend(title='Region')
plt.ylim(0, .5)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()